In [9]:
import numpy as np
import pandas as pd
import array

In [2]:
train_data = pd.read_csv("adult.data", sep=",")

test_data = pd.read_csv("adult.test", sep=",")

In [3]:
temp_mass = np.array([0 if x == 'Male' else 1 for x in train_data['sex'].values])
train_data['sex'] = temp_mass
#White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black
temp_mass = np.array([0 if x == 'Other' else 1 if x == 'Asian-Pac-Islander' else 2 if x == 'Black' else 3 if x == 'Amer-Indian-Eskimo' else 3 if x == 'White' else 0 for x in train_data['race'].values])
train_data['race'] = temp_mass

exp1_data_train = train_data[['age','hours-per-week','education-num','race','sex']]

temp_mass = np.array([0 if x == '<=50K' else 1 for x in train_data['pay'].values])
train_data['pay'] = temp_mass

In [4]:
train_data.sample(15)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,pay
6329,25,Private,25249,Some-college,10,Never-married,Adm-clerical,Not-in-family,3,1,0,0,40,United-States,0
27632,24,Private,72119,Bachelors,13,Never-married,Adm-clerical,Own-child,3,0,2202,0,30,United-States,0
18038,31,Private,195136,Assoc-acdm,12,Divorced,Other-service,Not-in-family,3,1,0,0,32,United-States,0
26402,22,Private,214635,HS-grad,9,Never-married,Machine-op-inspct,Not-in-family,3,0,0,0,42,United-States,0
23764,48,Private,195491,HS-grad,9,Divorced,Sales,Not-in-family,3,1,0,0,30,United-States,0
15784,20,Private,122166,Some-college,10,Never-married,Adm-clerical,Not-in-family,3,1,0,0,40,United-States,0
24711,40,Self-emp-not-inc,129298,HS-grad,9,Married-civ-spouse,Other-service,Husband,3,0,0,0,45,United-States,0
11112,33,Private,96128,11th,7,Never-married,Craft-repair,Own-child,3,0,0,0,40,United-States,0
26280,38,Private,34378,HS-grad,9,Married-civ-spouse,Sales,Husband,3,0,0,0,55,United-States,0
31957,26,Private,473625,HS-grad,9,Married-civ-spouse,Other-service,Wife,3,1,0,0,30,United-States,0


In [26]:
#Создаём массив для хранения значений в вершинах деревьев
level = 3
parameters = ['age','education-num','race','sex','hours-per-week']

tree_value = array.array('i',[0])
tree_col = ['']
for i in range(2**level-2):
    tree_value.append(0)
    tree_col.append('')

In [46]:
for p in parameters:
        loc_col = train_data[p]
        max_val = loc_col.max()
        min_val = loc_col.min()
        for i in range(min_val,max_val):
            print(i)

17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [47]:
#Создаём функцию для определения характеристик вершин
def ramification(lev, num, data, level, parameters):
    num1 = 2**lev - 2 + (num - 2**(lev - 1) + 1) * 2 + 1
    num2 = num1 + 1
    min_Q_glob = 100000000000000000
    for p in parameters:
        loc_col_pay = data['pay']
        loc_col = data[p]
        max_val = loc_col.max()
        min_val = loc_col.min()
        min_Q = 100000000000000
        for i in range(min_val,max_val):
            Hxl=0
            Hxr=0
            Xl=0
            Xr=0
            for j in range(len(loc_col.index)):
                if (i > loc_col[j] and loc_col_pay[j] == 1):
                    Hxl = Hxl + 1
                if (i <= loc_col[j] and loc_col_pay[j] == 0):
                    Hxr = Hxr + 1
                if (i > loc_col [j]):
                    Xl = Xl + 1
                if (i > loc_col [j]):
                    Xr = Xr + 1
            Q = Xl/(len(loc_col.index)+1)*Hxl/(Xl+1)+Xr/(len(loc_col.index)+1)*Hxr/(Xr+1)
            if (Q < min_Q):
                min_Q = Q
                min_par = i
        if(min_Q < min_Q_glob):
            min_Q_glob = min_Q
            min_par_glob = p
            min_par_num_glob = min_par
    #Нашли параметр и критерй для разделения, теперь надо разделить значения и создать 2 таблицы
    if(level > lev):
        mask = data[min_par_glob] >= min_par_num_glob
    
        data1 = data[mask]
        data2 = data[~mask]
    
        tree_value_1, tree_col_1 = ramification(lev + 1, num1, data1, level, parameters)
        tree_value_2, tree_col_2 = ramification(lev + 1, num2, data2, level, parameters)
    
    #Создание массива для отправки
    
    tree_value = array.array('i',[0])
    tree_col = ['']
    for i in range(2**level-2):
        tree_value.append(0)
        tree_col.append('')
    
    tree_value[num] = min_par_num_glob
    tree_col[num] = min_par_glob
    
    if(level > lev):
        tree_value = tree_value + tree_value_1 + tree_value_2
        tree_col = tree_col + tree_col_1 + tree_col_2
    
    return tree_value, tree_col

tree_value, tree_col = ramification(1, 0, train_data, level, parameters)

tree_col.head()

TypeError: 'float' object cannot be interpreted as an integer